In [76]:
num_epochs = 10
batch_size = 32
best_accuracy = 0
input_size=288
hidden_size=64
num_layers=2
num_classes =1
sequence_length =1

In [77]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from IPython.display import clear_output
import time

In [78]:
class MyGRU(nn.Module):
    def __init__(self, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes, sequence_length=sequence_length):
        super(MyGRU, self).__init__()
        self.hidden_size  = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length, hidden_size * sequence_length)
        self.fc2 = nn.Linear(hidden_size * sequence_length, num_classes)
    
    def forward(self, x):
        x = x.unsqueeze(1)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out,_ = self.gru(x, h0)
        out = out[:, -1, :]  # 取最後一個時間步的輸出
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [80]:
model = torch.load('E:/專題/predict_reward_model/GRU_predictor_model_20250315_012915.pth')
model.to(device)
loss_criterion  = nn.MSELoss()
model.eval()

C:\Users\timy3\AppData\Local\Temp\ipykernel_27460\4149191094.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('E:/專題/predict_reward_model/GRU_predictor

MyGRU(
  (gru): GRU(288, 64, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [81]:
class MyDataset(Dataset):
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        
        self.columns = pd.read_csv(self.file_path, nrows=0).columns.str.strip().str.replace('\xa0', ' ').tolist()
        self.train_columns = [col for col in self.columns if col != 'Reward']  # 排除 'Reward' 欄位
        self.num_samples = sum(1 for _ in open(self.file_path, encoding='utf-8')) - 1  # 計算總樣本數
        
    def __getitem__(self, idx):
        chunk_start = idx // self.chunk_size * self.chunk_size
        df = pd.read_csv(self.file_path, skiprows=chunk_start + 1, nrows=self.chunk_size, header=None, encoding='utf-8')  # 跳過標題和之前的行

        df.columns = self.columns
        
        # 確保 Reward 欄位存在
        if 'Reward' not in df.columns:
            raise KeyError(f"Chunk starting at row {chunk_start + 1} does not contain 'Reward' column.")
        
            
        sample_idx = idx % self.chunk_size
        
        train_data = df[self.train_columns].iloc[sample_idx].values  # 提取除了 'Reward' 之外的欄位
        
        value_data = df['Reward'].iloc[sample_idx]  # 提取 'Reward' 欄位
        
        return torch.tensor(train_data, dtype=torch.float32), torch.tensor(value_data, dtype=torch.float32)
        
    def __len__(self):
        return self.num_samples

In [82]:
"""
讀取csv
"""
file_path = 'E:/專題/data/2022/PredictorData.csv'
dataset = MyDataset(file_path, chunk_size=50000)  # 每次只加載 50000 行
test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [83]:
current_loss = 0
times = 0
total = 0

for epoch in range(num_epochs):
    for train_data, value_data in test_dataloader:
        times += 1
        train_data = train_data.to(device)
        value_data = value_data.to(device)
        output = model(train_data)
        value_data = value_data.unsqueeze(1)
        loss = loss_criterion(output, value_data)
        current_loss = loss
        total += current_loss
        avg_loss = total/ times
        
        
        clear_output()
        print(f"At epoch: {epoch}, loss: {current_loss}")
        print("avg_loss:",avg_loss)




KeyboardInterrupt: 